#!pip install mediapipe-model-maker


import os
import pathlib
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

path=str(pathlib.Path().resolve())+"/"

from genericpath import isfile
from os import listdir, mkdir
import shutil

if not os.path.exists(path+"Piedra_Papel_Tijera_Formated"):
    gestos=([],[],[],[])
    directories = [f for f in listdir(path+"Piedra_Papel_Tijeras")]
    for i in directories:
        for index,(dirpath, dirnames, filenames) in enumerate(os.walk(path+"Piedra_Papel_Tijeras/"+i+"/")):
            if len(dirnames)==0:
                for filename in filenames:
                    gestos[index-1].append(dirpath+"/"+filename)
    mkdir(path+"Piedra_Papel_Tijera_Formated")
    mkdir(path+"Piedra_Papel_Tijera_Formated/None")
    mkdir(path+"Piedra_Papel_Tijera_Formated/Piedra")
    mkdir(path+"Piedra_Papel_Tijera_Formated/Papel")
    mkdir(path+"Piedra_Papel_Tijera_Formated/Tijeras")

# Comprueba si el directorio existe
    if not os.path.exists(path):
    # Si no existe, créalo
        os.makedirs(path)
    for index,i in enumerate(gestos):
        for j in i:
            split=i[index].split("/")
            split.remove(split[-3])
            shutil.copy(j,path+"Piedra_Papel_Tijera_Formated/"+split[-2].title())


data = gesture_recognizer.Dataset.from_folder(
    dirname=os.path.join(path,"Piedra_Papel_Tijera_Formated"),
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train,
    validation_data=validation,
    options=options
)

from google.colab import files
model.export_model()
files.download('exported_model/gesture_recognizer.task')

hparams = gesture_recognizer.HParams(learning_rate=0.003, export_dir="exported_model_2")
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.2)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model_2 = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

loss, accuracy = model.evaluate(test_data)
print(f"Test loss:{loss}, Test accuracy:{accuracy}")
loss, accuracy = model_2.evaluate(test_data)
print(f"Test loss:{loss}, Test accuracy:{accuracy}")

from google.colab import files
model_2.export_model()
files.download('exported_model/gesture_recognizer.task')

In [1]:
import pathlib


path=str(pathlib.Path().resolve())+"/"

In [3]:
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

model_path = path+'gesture_recognizer.task'
base_options = python.BaseOptions(model_asset_path=model_path)